In [93]:
pip install -r requirements.txt


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: /opt/homebrew/Cellar/jupyterlab/4.0.6/libexec/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


# LCEL & Runnables

### Quick Notes
- Expose schematic information about their input, output and config through their `.input_schema` property, the `.output_schema` property and the `.config_schema` method
- LCEL is a declarative way to compose Runnables into chains - any chain constructed with LCEL will also always automatically have sync, async, batch, and streaming support

### Main Primitives
#### 1. RunnableSequence
- Invokes a series of Runnables sequentially
- output of the previous runnable will be input to the next runnable in the chain
- can use pipe operator to construct or by passing a list of RunnableSequence
- 
#### 2. Runnable Parallel
- Invokes runnables concurrently, providing the same input to each. Construct using dict literal within a sequence or by passing a dict to RunnableParallel

### Additional Methods
- can be used to modify behavior such as retry policy, lifecycle listeners, make them configurable 

### Debugging
- You can use `set_debug` from `from langchain_core.globals import set_debug`. set_debug(True)
- You can pass existing or custom callbacks to any give chain too:

```from langchain_core.tracers import ConsoleCallbackHandler

chain.invoke(
    ...,
    config={'callbacks': [ConsoleCallbackHandler()]}
)```

### Passing Data through
- RunnablePassthrough allows to pass inputs unchanged or with the addition of extra keys. This typically is used in conjuction with RunnableParallel to assign data to a new key in the map.
- RunnablePassthrough() called on it’s own, will simply take the input and pass it through.

In [94]:
from langchain.schema.runnable import RunnableParallel, RunnablePassthrough

runnable = RunnableParallel(
    passed=RunnablePassthrough(),
    extra=RunnablePassthrough.assign(mult=lambda x: x["num"] * 3),
    modified=lambda x: x["num"] + 1,
)

runnable.invoke({"num": 1})

{'passed': {'num': 1}, 'extra': {'num': 1, 'mult': 3}, 'modified': 2}

- The `passed` key was called with RunnablePassthrough() and so it passed on {'num': 1}.

- In the second line with the key `extra`, we used RunnablePastshrough.assign with a lambda that multiplies the passed integer value by 3. So, `extra` was set with {'num': 1, 'mult': 3} which is the original value with the `'mult'` key added.

- Finally, we set a third key called `modified` in the map which uses a labmda to set a single value adding 1 to the num, which resulted in modified key with the value of 2.

### Run custom functions

In [95]:
from operator import itemgetter

from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnableLambda


def length_function(text):
    return len(text)


def _multiple_length_function(text1, text2):
    return len(text1) * len(text2)


def multiple_length_function(_dict):
    return _multiple_length_function(_dict["text1"], _dict["text2"])


prompt = ChatPromptTemplate.from_template("what is {a} + {b}")
model = ChatOpenAI()

chain1 = prompt | model

chain = (
    {
        "a": itemgetter("foo") | RunnableLambda(length_function),
        "b": {"text1": itemgetter("foo"), "text2": itemgetter("bar")}
        | RunnableLambda(multiple_length_function),
    }
    | prompt
    | model
)

#### NOTE
All inputs to these functions need to be a **SINGLE** argument. If you have a function that accepts multiple arguments, you should write a wrapper that accepts a single input and unpacks it into multiple argument.

In [96]:
chain.invoke({"foo": "bar", "bar": "gah"})

AIMessage(content='3 + 9 equals 12.')

- We use `itemgetter` to extract the input arguments
- `"a"` becomes `3` because `itemgetter` gets the value of `"foo"` and that is passed through pipe operator to the RunnableLamba which passes the value to the `length_function` which returns length of `"bar"` which is `3`
- `"b"` is 9 because we pass the dictionary of `{"text1": "bar", "text2": "gah"}` to the RunnableLambda that passes it into the `multiple_length_function`
- the dictionary of `{"a":3, "b": 9}` is then passed as output to the prompt usuing the pipe operator (`|`)
- the output of the prompt again is passed to the model
  

### Accepting Runnable Configs

In [97]:
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableConfig
import json


def parse_or_fix(text: str, config: RunnableConfig):
    fixing_chain = (
        ChatPromptTemplate.from_template(
            "Fix the following text:\n\n```text\n{input}\n```\nError: {error}"
            " Don't narrate, just respond with the fixed data."
        )
        | ChatOpenAI()
        | StrOutputParser()
    )
    for _ in range(3):
        try:
            return json.loads(text)
        except Exception as e:
            text = fixing_chain.invoke({"input": text, "error": e}, config)
    return "Failed to parse"


from langchain.callbacks import get_openai_callback

with get_openai_callback() as cb:
    output = RunnableLambda(parse_or_fix).invoke(
        "{foo: bar}", {"tags": ["my-tag"], "callbacks": [cb]}
    )
    print(output)
    print(cb)

{'foo': 'bar'}
Tokens Used: 65
	Prompt Tokens: 56
	Completion Tokens: 9
Successful Requests: 1
Total Cost (USD): $0.00010200000000000001


- We use RunnableLambda with a custom function that allows us a retry protocol for calling OpenAI models

## Dynamic Routing based on Input

**Two ways to perform routing:**
1. Using a `RunnableBranch`.
2. Writing custom factory function that takes the input of a previous step and returns a runnable. Importantly, this should return a runnable and NOT actually execute.

### RunnableBranch

In [98]:
import os
import dotenv

dotenv.load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

In [99]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.schema.output_parser import StrOutputParser

In [101]:
chain = (
    PromptTemplate.from_template(
        """Given the user question below, classify it as either being about `Bodybuilding`, `Jungian Psychology`, or `Other`.

        Do not respond with more than one word.

        <question>
        {question}
        </question>

        Classification:"""
    )
    | ChatOpenAI()
    | StrOutputParser()
)

In [102]:
chain.invoke({"question": "How do I bench press properly?"})

'Bodybuilding'

In [103]:
chain.invoke({"question": "How do I do active imagination?"})

'Jungian Psychology'

In [104]:
body_building_chain =  ( PromptTemplate.from_template(
        """You are an expert in bodybuilding. \
        Always answer questions starting with "As Dr. Mike Israetel from Renaissance Periodization told me". \
        Respond to the following question:
        
        Question: {question}
        Answer:"""
    )
    | ChatOpenAI()
)
jungian_chain = (PromptTemplate.from_template(
    """
    You are an expert in Jungian Psychology and Theory. \
    Always answer questions starting with "As Dr. C.G. Jung would say". \
    Respond to the following question:

    Question: {question}
    Answer:
    """) | ChatOpenAI())

In [105]:
general_chain = (
    PromptTemplate.from_template(
        """Respond to the following question:

Question: {question}
Answer:"""
    )
    | ChatOpenAI()
)

In [106]:
from langchain.schema.runnable import RunnableBranch

branch = RunnableBranch(
    (lambda x: "bodybuilding" in x["topic"].lower(), body_building_chain),
    (lambda x: "jungian psychology" in x["topic"].lower(), jungian_chain),
    general_chain,
)

In [107]:
full_chain = {"topic": chain, "question": lambda x: x["question"]} | branch

In [108]:
full_chain.invoke({"question": "how do I interpret my dreams?"})

AIMessage(content='As Dr. C.G. Jung would say, to interpret your dreams, it is important to approach them with an open and curious mindset. Dreams, according to Jung, are a window into the unconscious and can provide valuable insights into your psyche. \n\nFirstly, record your dreams in a journal immediately upon waking up, as details can easily be forgotten. Pay attention to the emotions, symbols, and themes present in your dreams. \n\nNext, recognize that dreams often contain both personal and collective symbols. Personal symbols are unique to each individual and are influenced by personal experiences and memories. Collective symbols, on the other hand, are shared by humanity and can be found across different cultures and time periods. \n\nTo interpret your dreams, start by analyzing the personal symbols. Reflect on what these symbols mean to you personally, considering your memories and experiences associated with them. Explore the emotions evoked by these symbols and try to connect

In [109]:
full_chain.invoke({"question": "whats 2 + 2"})

AIMessage(content='The answer to 2 + 2 is 4.')

### Routing with Custom Function

In [110]:
def route(info):
    if "bodybuilding" in info["topic"].lower():
        return body_building_chain
    elif "jungian psychology" in info["topic"].lower():
        return jungian_chain
    else:
        return general_chain

In [111]:
from langchain.schema.runnable import RunnableLambda

full_chain = {"topic": chain, "question": lambda x: x["question"]} | RunnableLambda(
    route
)

In [112]:
full_chain.invoke({"question": "how do I grow my traps?"})

AIMessage(content="As Dr. Mike Israetel from Renaissance Periodization told me, to grow your traps, you can focus on incorporating specific exercises that target this muscle group. Some effective exercises include barbell shrugs, dumbbell shrugs, upright rows, and trap bar deadlifts. It's important to ensure proper form and gradually increase the weights as you progress. Additionally, incorporating a variety of rep ranges, including both heavy weights for lower reps and lighter weights for higher reps, can help stimulate muscle growth in the traps. Remember to prioritize progressive overload, prioritize proper nutrition and recovery, and be patient as muscle growth takes time.")

In [113]:
full_chain.invoke({"question": "what does alchemy have to do with dreams?"})

AIMessage(content="As Dr. C.G. Jung would say, alchemy and dreams are interconnected in the realm of the unconscious. Alchemy, traditionally seen as the precursor to modern chemistry, also held a symbolic significance in psychological terms. Jung considered alchemy to be a symbolic system that mirrored the individuation process, which is the journey towards self-realization and wholeness.\n\nIn the context of dreams, alchemy can be seen as a metaphorical language used by the unconscious to convey psychological transformation and inner development. Dreams often contain symbolic elements that parallel alchemical symbols such as the union of opposites, the transformation of base materials into gold, and the concept of the Philosopher's Stone.\n\nJust as alchemists worked with various substances and processes to transform lead into gold, dreams can be seen as a psychological laboratory where the unconscious works towards transforming the psyche. Alchemical symbols present in dreams can rep

## Binding runtime arguments
- we can use `Runnable.bind()` to pass arguments as constants so that we can have access to them even within a runnable sequence where the argument is not part of the output of preceding runnables in the sequence

In [114]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough

In [115]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Write out 4 flashcard questions with their options based on the topic given below.. Use the format\n\QUESTIONS:...\nOPTIONS:...\nSOLUTION:...\n\n",
        ),
        ("human", "{topic}"),
    ]
)
model = ChatOpenAI(temperature=0)
runnable = (
    {"topic": RunnablePassthrough()} | prompt | model | StrOutputParser()
)

print(runnable.invoke("the cuban missile crisis"))

QUESTION: Who was the leader of the Soviet Union during the Cuban Missile Crisis?
OPTIONS: 
a) Nikita Khrushchev
b) Vladimir Putin
c) Joseph Stalin
d) Mikhail Gorbachev
SOLUTION: a) Nikita Khrushchev

QUESTION: Which country discovered the presence of Soviet missiles in Cuba?
OPTIONS: 
a) United States
b) Canada
c) Mexico
d) Brazil
SOLUTION: a) United States

QUESTION: How did the United States respond to the discovery of Soviet missiles in Cuba?
OPTIONS: 
a) Imposed a naval blockade
b) Launched a military invasion
c) Sent a diplomatic envoy
d) Ignored the situation
SOLUTION: a) Imposed a naval blockade

QUESTION: How did the Cuban Missile Crisis end?
OPTIONS: 
a) The United States and Soviet Union engaged in a nuclear war
b) The United States and Soviet Union reached a peaceful resolution
c) Cuba launched a missile attack on the United States
d) The United Nations intervened and resolved the crisis
SOLUTION: b) The United States and Soviet Union reached a peaceful resolution


In [116]:
runnable = (
    {"topic": RunnablePassthrough()}
    | prompt
    | model.bind(stop="SOLUTION")
    | StrOutputParser()
)
print(runnable.invoke("the cuban missile crisis"))

QUESTION: Who was the leader of the Soviet Union during the Cuban Missile Crisis?
OPTIONS: 
a) Nikita Khrushchev
b) Vladimir Putin
c) Joseph Stalin
d) Mikhail Gorbachev



### Attach OpenAI Functions

In [118]:
function = {
    "name": "return_questions",
    "description": "extracts questions from raw text",
    "parameters": {
        "type": "object",
        "properties": {
            "raw_text": {
                "type": "string",
                "description": "The raw text of flashcard questions",
            },
            "questions": {
                "type": "array",
                "description": "array of questions",
                "items": {
                    "type": "string",
                    "description": "question string"
                }
            },
        },
        "required": ["equation", "solution"],
    },
}

In [119]:
# Need gpt-4 to solve this one correctly
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Write out flashcard questions to the following topic then return the list of questions.",
        ),
        ("human", "{topic}"),
    ]
)
model = ChatOpenAI(model="gpt-4", temperature=0).bind(
    function_call={"name": "return_questions"}, functions=[function]
)
runnable = {"topic": RunnablePassthrough()} | prompt | model
runnable.invoke("world war 2")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{\n  "questions": [\n    "What event sparked the beginning of World War 2?",\n    "Who were the Axis Powers in World War 2?",\n    "Who were the Allied Powers in World War 2?",\n    "What was the Holocaust?",\n    "What was the significance of the Battle of Stalingrad?",\n    "What was the role of the United States in World War 2?",\n    "What was the Manhattan Project?",\n    "What was the significance of D-Day?",\n    "What were the Nuremberg Trials?",\n    "What was the outcome of World War 2?",\n    "Who was the leader of Nazi Germany during World War 2?",\n    "What was the role of Japan in World War 2?",\n    "What was the impact of World War 2 on the world?",\n    "What was the role of women during World War 2?",\n    "What was the Blitz?",\n    "What was the significance of the Battle of Midway?",\n    "What was the Atlantic Charter?",\n    "What was the Yalta Conference?",\n    "What was the Potsdam Confe

## Fallbacks
- we can use fallbacks at the runnable level

In [120]:
from langchain.chat_models import ChatAnthropic, ChatOpenAI
import dotenv
dotenv.load_dotenv()


True

In [121]:
os.environ["ANTHROPIC_API_KEY"] = os.getenv("ANTHROPIC_API_KEY")

In [122]:
from unittest.mock import patch

import httpx
from openai import RateLimitError

request = httpx.Request("GET", "/")
response = httpx.Response(200, request=request)
error = RateLimitError("rate limit", response=response, body="")

In [123]:
# Note that we set max_retries = 0 to avoid retrying on RateLimits, etc
openai_llm = ChatOpenAI(max_retries=0)
anthropic_llm = ChatAnthropic()
llm = openai_llm.with_fallbacks([anthropic_llm])

In [124]:
# Let's use just the OpenAI LLm first, to show that we run into an error
with patch("openai.resources.chat.completions.Completions.create", side_effect=error):
    try:
        print(openai_llm.invoke("Why did the chicken cross the road?"))
    except RateLimitError:
        print("Hit error")

Hit error


In [125]:
# Now let's try with fallbacks to Anthropic
with patch("openai.resources.chat.completions.Completions.create", side_effect=error):
    try:
        print(llm.invoke("Why did the chicken cross the road?"))
    except RateLimitError:
        print("Hit error")

content=' I don\'t have enough context to determine the chicken\'s true motivation, but the classic punchline is: "To get to the other side!" It\'s an anti-joke playing on the double meaning of "the other side" referring to either the other side of the road, or the afterlife. Without more details, I\'d just be clucking in the dark trying to explain this chicken\'s reasoning.'


In [127]:
from langchain.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You're a nice assistant who always includes a compliment in your response",
        ),
        ("human", "Why did the {animal} cross the road"),
    ]
)
chain = prompt | llm
with patch("openai.resources.chat.completions.Completions.create", side_effect=error):
    try:
        print(chain.invoke({"animal": "kangaroo"}))
    except RateLimitError:
        print("Hit error")

content=" I don't know, why did the kangaroo cross the road?"


In [128]:
# First let's create a chain with a ChatModel
# We add in a string output parser here so the outputs between the two are the same type
from langchain.schema.output_parser import StrOutputParser

chat_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You're a nice assistant who always includes a compliment in your response",
        ),
        ("human", "Why did the {animal} cross the road"),
    ]
)
# Here we're going to use a bad model name to easily create a chain that will error
chat_model = ChatOpenAI(model_name="gpt-fake")
bad_chain = chat_prompt | chat_model | StrOutputParser()

In [129]:
# Now lets create a chain with the normal OpenAI model
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate

prompt_template = """Instructions: You should always include a compliment in your response.

Question: Why did the {animal} cross the road?"""
prompt = PromptTemplate.from_template(prompt_template)
llm = OpenAI()
good_chain = prompt | llm

In [130]:
# We can now create a final chain which combines the two
chain = bad_chain.with_fallbacks([good_chain])
chain.invoke({"animal": "turtle"})

"\n\nAnswer: The turtle crossed the road to get to the other side! That's a great question, by the way - you have a great sense of humor!"